In [85]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, utils
import matplotlib.pyplot as plt
from skimage import io, transform
from PIL import Image
import sklearn.metrics as mt
from os import path
import copy
from importlib import reload
import itertools
import src.trainer as tr
import src.dataset as ds
import src.predict as pr
reload(tr)
reload(ds)
reload(pr)
# %matplotlib inline

<module 'src.predict' from '/Users/habbes/code/class/bd4h/chexpert/src/predict.py'>

In [19]:
dummy = pd.read_csv('./data/dummy_image_paths.csv')

In [20]:
dummy.head()

,Path
0,CheXpert-v1.0/valid/patient00000/study1/view1_...
1,CheXpert-v1.0/valid/patient00000/study1/view2_...
2,CheXpert-v1.0/valid/patient00000/study2/view1_...


In [22]:
dummy.sort_values('Path')

,Path
0,CheXpert-v1.0/valid/patient00000/study1/view1_...
1,CheXpert-v1.0/valid/patient00000/study1/view2_...
2,CheXpert-v1.0/valid/patient00000/study2/view1_...


In [23]:
valid = pd.read_csv(ds.VALID_CSV)
valid.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/valid/patient64541/study1/...,Male,73,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CheXpert-v1.0-small/valid/patient64542/study1/...,Male,70,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,CheXpert-v1.0-small/valid/patient64542/study1/...,Male,70,Lateral,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,CheXpert-v1.0-small/valid/patient64543/study1/...,Male,85,Frontal,AP,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CheXpert-v1.0-small/valid/patient64544/study1/...,Female,42,Frontal,AP,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
valid.

Path                          234
Sex                           234
Age                           234
Frontal/Lateral               234
AP/PA                         202
No Finding                    234
Enlarged Cardiomediastinum    234
Cardiomegaly                  234
Lung Opacity                  234
Lung Lesion                   234
Edema                         234
Consolidation                 234
Pneumonia                     234
Atelectasis                   234
Pneumothorax                  234
Pleural Effusion              234
Pleural Other                 234
Fracture                      234
Support Devices               234
dtype: int64

In [29]:
valid['study'] = valid['Path'].apply(lambda p: path.dirname(p))

In [34]:
valid.sort_values('study', inplace=True)
valid.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,study
0,CheXpert-v1.0-small/valid/patient64541/study1/...,Male,73,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CheXpert-v1.0-small/valid/patient64541/study1
1,CheXpert-v1.0-small/valid/patient64542/study1/...,Male,70,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,CheXpert-v1.0-small/valid/patient64542/study1
2,CheXpert-v1.0-small/valid/patient64542/study1/...,Male,70,Lateral,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,CheXpert-v1.0-small/valid/patient64542/study1
3,CheXpert-v1.0-small/valid/patient64543/study1/...,Male,85,Frontal,AP,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CheXpert-v1.0-small/valid/patient64543/study1
4,CheXpert-v1.0-small/valid/patient64544/study1/...,Female,42,Frontal,AP,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CheXpert-v1.0-small/valid/patient64544/study1


In [37]:
val = ds.get_val_loader()

In [38]:
next(iter(val))

IDX 64
IDX 0
IDX 128
IDX 192
IDX 65
IDX 129
IDX 193
IDX 1
IDX 66
IDX 130
IDX 2
IDX 194
IDX 131
IDX 3
IDX 67
IDX 195
IDX 132
IDX 68
IDX 4
IDX 196
IDX 133
IDX 69
IDX 197
IDX 5
IDX 134
IDX 70
IDX 6
IDX 198
IDX 71
IDX 135
IDX 7
IDX 199
IDX 72
IDX 136
IDX 8
IDX 200
IDX 73
IDX 137
IDX 201
IDX 9
IDX 74
IDX 138
IDX 202
IDX 10
IDX 139
IDX 75
IDX 203
IDX 140
IDX 76
IDX 11
IDX 204
IDX 141
IDX 12
IDX 77
IDX 205
IDX 78
IDX 142
IDX 13
IDX 206
IDX 143
IDX 79
IDX 14
IDX 207
IDX 80
IDX 144
IDX 15
IDX 81
IDX 208
IDX 16
IDX 145
IDX 82
IDX 209
IDX 17
IDX 146
IDX 83
IDX 210
IDX 147
IDX 18
IDX 211
IDX 84
IDX 148
IDX 19
IDX 212
IDX 85
IDX 20
IDX 149
IDX 213
IDX 86
IDX 150
IDX 21
IDX 214
IDX 22
IDX 87
IDX 151
IDX 215
IDX 23
IDX 216
IDX 152
IDX 88
IDX 217
IDX 24
IDX 153
IDX 89
IDX 218
IDX 154
IDX 25
IDX 219
IDX 90
IDX 155
IDX 26
IDX 220
IDX 156
IDX 27
IDX 91
IDX 157
IDX 221
IDX 28
IDX 92
IDX 29
IDX 222
IDX 158
IDX 223
IDX 159
IDX 30
IDX 93
IDX 224
IDX 160
IDX 31
IDX 94
IDX 225
IDX 161
IDX 32
IDX 95
IDX 162
IDX

[tensor([[[[-1.9980, -1.9980, -1.9980,  ..., -1.9295, -1.9295, -1.9295],
           [-2.0152, -1.9980, -1.9980,  ..., -1.9295, -1.9295, -1.9295],
           [-2.0152, -2.0152, -1.9980,  ..., -1.9295, -1.9295, -1.9295],
           ...,
           [ 1.9407,  1.9407,  1.9407,  ..., -0.3198, -0.5082, -1.4672],
           [ 1.9578,  1.9920,  1.9578,  ..., -0.3883, -0.8678, -1.6042],
           [ 1.9920,  1.9920,  1.9920,  ..., -0.4054, -1.0733, -1.7240]],
 
          [[-1.9132, -1.9132, -1.9132,  ..., -1.8431, -1.8431, -1.8431],
           [-1.9307, -1.9132, -1.9132,  ..., -1.8431, -1.8431, -1.8431],
           [-1.9307, -1.9307, -1.9132,  ..., -1.8431, -1.8431, -1.8431],
           ...,
           [ 2.1134,  2.1134,  2.1134,  ..., -0.1975, -0.3901, -1.3704],
           [ 2.1310,  2.1660,  2.1310,  ..., -0.2675, -0.7577, -1.5105],
           [ 2.1660,  2.1660,  2.1660,  ..., -0.2850, -0.9678, -1.6331]],
 
          [[-1.6824, -1.6824, -1.6824,  ..., -1.6127, -1.6127, -1.6127],
           [-

In [44]:
ts = ds.get_test_loader(ds.VALID_CSV, data_dir='./data')

In [47]:
inputs, paths = next(iter(ts))

In [48]:
inputs

tensor([[[[-1.9295, -1.9295, -1.9295,  ..., -2.0323, -2.0323, -2.0323],
          [-1.9295, -1.9295, -1.9295,  ..., -2.0323, -2.0323, -2.0323],
          [-1.9295, -1.9295, -1.9295,  ..., -2.0323, -2.0323, -2.0323],
          ...,
          [ 1.7009,  1.6667,  1.6838,  ...,  1.4954,  1.4954,  1.4269],
          [ 1.7009,  1.6838,  1.6324,  ...,  1.5297,  1.4612,  1.4954],
          [ 1.6838,  1.6495,  1.5639,  ...,  1.4954,  1.4269,  1.5297]],

         [[-1.8431, -1.8431, -1.8431,  ..., -1.9482, -1.9482, -1.9482],
          [-1.8431, -1.8431, -1.8431,  ..., -1.9482, -1.9482, -1.9482],
          [-1.8431, -1.8431, -1.8431,  ..., -1.9482, -1.9482, -1.9482],
          ...,
          [ 1.8683,  1.8333,  1.8508,  ...,  1.6583,  1.6583,  1.5882],
          [ 1.8683,  1.8508,  1.7983,  ...,  1.6933,  1.6232,  1.6583],
          [ 1.8508,  1.8158,  1.7283,  ...,  1.6583,  1.5882,  1.6933]],

         [[-1.6127, -1.6127, -1.6127,  ..., -1.7173, -1.7173, -1.7173],
          [-1.6127, -1.6127, -

In [52]:
np.array(paths)

array(['./data/CheXpert-v1.0-small/valid/patient64541/study1/view1_frontal.jpg',
       './data/CheXpert-v1.0-small/valid/patient64542/study1/view1_frontal.jpg',
       './data/CheXpert-v1.0-small/valid/patient64542/study1/view2_lateral.jpg',
       './data/CheXpert-v1.0-small/valid/patient64543/study1/view1_frontal.jpg',
       './data/CheXpert-v1.0-small/valid/patient64544/study1/view1_frontal.jpg',
       './data/CheXpert-v1.0-small/valid/patient64545/study1/view1_frontal.jpg',
       './data/CheXpert-v1.0-small/valid/patient64546/study1/view1_frontal.jpg',
       './data/CheXpert-v1.0-small/valid/patient64547/study1/view1_frontal.jpg',
       './data/CheXpert-v1.0-small/valid/patient64547/study1/view2_frontal.jpg',
       './data/CheXpert-v1.0-small/valid/patient64547/study1/view3_lateral.jpg',
       './data/CheXpert-v1.0-small/valid/patient64548/study1/view1_frontal.jpg',
       './data/CheXpert-v1.0-small/valid/patient64549/study1/view1_frontal.jpg',
       './data/CheXpert-v1.0

In [56]:
np.concatenate((np.array(['a', 'b']), np.array(['c', 'd'])))

array(['a', 'b', 'c', 'd'], dtype='<U1')

In [57]:
trainer = tr.Trainer(max_train_samples=1, epochs=1)
trainer

Training using options arch densenet finetune False


/Users/habbes/anaconda3/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [58]:
trainer.train()

Epoch 0/0
----------
train Loss: 49.2671
val Loss: 0.7347

Training complete in 1m 15s
Least loss: 0.734742


{'losses': {'train': [49.26709747314453], 'val': [0.734742050496941]},
 'model': DenseNet(
   (features): Sequential(
     (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
     (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu0): ReLU(inplace)
     (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
     (denseblock1): _DenseBlock(
       (denselayer1): _DenseLayer(
         (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (relu1): ReLU(inplace)
         (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (relu2): ReLU(inplace)
         (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       )
       (denselayer2): _DenseLayer(
         (norm1): BatchNorm2d

In [59]:
res = trainer.evaluate()

AVG AUC 0.47752981924875676


In [61]:
labels, preds = res

In [63]:
t1, t2 = preds[0], preds[1]

In [64]:
t1

array([0.5237978 , 0.44100636, 0.5758407 , 0.4208122 , 0.6259521 ],
      dtype=float32)

In [65]:
t2

array([0.4933868 , 0.4114361 , 0.5749687 , 0.34154075, 0.608173  ],
      dtype=float32)

In [68]:
np.max(np.vstack((t1, t2)), axis=0)

array([0.5237978 , 0.44100636, 0.5758407 , 0.4208122 , 0.6259521 ],
      dtype=float32)

In [70]:
tt = np.vstack((t1, t2))
tt

array([[0.5237978 , 0.44100636, 0.5758407 , 0.4208122 , 0.6259521 ],
       [0.4933868 , 0.4114361 , 0.5749687 , 0.34154075, 0.608173  ]],
      dtype=float32)

In [71]:
tp = ['p1', 'p2']

In [72]:
d = { 'p1': t1, 'p2': t2 }

In [81]:
next(iter(d.items()))

('p1', array([0.5237978 , 0.44100636, 0.5758407 , 0.4208122 , 0.6259521 ],
       dtype=float32))

In [82]:
['p1'] + list(t1)

['p1', 0.5237978, 0.44100636, 0.5758407, 0.4208122, 0.6259521]

In [91]:
print(d)
reload(pr)
df = pr.results_to_df(d)

{'p1': array([0.5237978 , 0.44100636, 0.5758407 , 0.4208122 , 0.6259521 ],
      dtype=float32), 'p2': array([0.4933868 , 0.4114361 , 0.5749687 , 0.34154075, 0.608173  ],
      dtype=float32)}
R ['p1', 0.5237978, 0.44100636, 0.5758407, 0.4208122, 0.6259521] [0.5237978  0.44100636 0.5758407  0.4208122  0.6259521 ]
R ['p2', 0.4933868, 0.4114361, 0.5749687, 0.34154075, 0.608173] [0.4933868  0.4114361  0.5749687  0.34154075 0.608173  ]


In [94]:
df.to_csv(index=False)

'Study,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion\np1,0.5237978100776672,0.4410063624382019,0.5758407115936279,0.42081218957901,0.6259521245956421\np2,0.49338680505752563,0.4114361107349396,0.574968695640564,0.34154075384140015,0.6081730127334595\n'